# CNN MNIST
## Mateusz Baran
### 155620

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torch.autograd import Variable

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [3]:
train_set = torchvision.datasets.MNIST(
    root='./data/MNIST',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.Resize((28, 28)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,)),
    ])
)

test_set = torchvision.datasets.MNIST(
    root='./data/MNIST',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.Resize((28, 28)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,)),
    ])
)

Failed to download (trying next):
HTTP Error 503: Service Unavailable



100%|██████████| 9912422/9912422 [00:04<00:00, 2218004.97it/s]


Extracting ./data/MNIST/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable



100%|██████████| 28881/28881 [00:00<00:00, 166638.73it/s]


Extracting ./data/MNIST/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable



100%|██████████| 1648877/1648877 [00:01<00:00, 1555579.41it/s]


Extracting ./data/MNIST/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable



100%|██████████| 4542/4542 [00:00<00:00, 2260653.70it/s]

Extracting ./data/MNIST/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/MNIST/raw


In [4]:
trainloader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True, drop_last=True)
testloader = torch.utils.data.DataLoader(test_set, batch_size=64, shuffle=True, drop_last=True)

In [6]:
sample=next(iter(train_set))
image=sample[0]
label=sample[1]
image.shape

torch.Size([1, 28, 28])

In [7]:
batch = next(iter(trainloader))
images, labels = batch

In [8]:
images.shape

torch.Size([64, 1, 28, 28])

In [9]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)
        
    def forward(self, t):
        t = F.relu(self.conv1(t))
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        t = F.relu(self.conv2(t))
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        t = F.relu(self.fc1(t.reshape(-1,12*4*4)))
        t = F.relu(self.fc2(t))
        t = self.out(t)
        return t

In [10]:
network = Network()
network.to(device)

error = nn.CrossEntropyLoss()

learning_rate = 0.001
optimizer = torch.optim.Adam(network.parameters(), lr=learning_rate)

In [11]:
network.cpu()

num_epochs = 5
count = 0

loss_list = []
iteration_list = []
accuracy_list = []

predictions_list = []
labels_list = []

for epoch in range(num_epochs):
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)

        train = Variable(images.view(64,1,28,28))
        labels = Variable(labels)

        outputs = network(train)
        loss = error(outputs, labels)

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()

        count += 1

        if not (count % 50):
            total = 0
            correct = 0

            for images, labels in testloader:
                images, labels = images.to(device), labels.to(device)
                labels_list.append(labels)
                test = Variable(images.view(64,1,28,28))
                outputs = network(test)
                predictions = torch.max(outputs, 1)[1].to(device)
                predictions_list.append(predictions)
                correct += (predictions == labels).sum()

            accuracy = correct / float(labels.size(0))
            loss_list.append(loss.data)
            iteration_list.append(count)
            accuracy_list.append(accuracy)

        if not (count % 500):
            print("Iteration: {}, Loss: {}, Accuracy: {}%".format(count, loss.data, accuracy))

Iteration: 500, Loss: 0.21440523862838745, Accuracy: 149.9375%
Iteration: 1000, Loss: 0.034073442220687866, Accuracy: 152.3125%
Iteration: 1500, Loss: 0.0828302651643753, Accuracy: 152.25%
Iteration: 2000, Loss: 0.025431601330637932, Accuracy: 153.234375%
Iteration: 2500, Loss: 0.02959035336971283, Accuracy: 152.515625%
Iteration: 3000, Loss: 0.0366499088704586, Accuracy: 153.796875%
Iteration: 3500, Loss: 0.03458021581172943, Accuracy: 153.703125%
Iteration: 4000, Loss: 0.009528356604278088, Accuracy: 153.953125%
Iteration: 4500, Loss: 0.025300676003098488, Accuracy: 153.78125%
